# SPAMMS Plots

Here we will go over the basics of gathering the output models from SPAMMS and plotting them in a meaningful way.

In [ ]:
# Lets import the necessary packages...
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib import cm, colors
%matplotlib notebook
plt.rcParams['figure.dpi'] = 100
import spamms as sp

In any given SPAMMS run, you could be calculating several different models so we need to specify which model we wish to visualize.  Lets start here:

In [ ]:
output_folder = 'Outputs/Demo_rapid_rotator/'
model_number = 'Model_0000'
He_abundance = '0.1'
CNO_abundance = '7.5'

Now that we have this, we can grab the necessary information from the input file.

In [ ]:
# grab the input file and use the builtin SPAMMS functions to read and interpret the input file
input_file = output_folder + 'input.txt'

fit_param_values, abund_param_values, line_list, io_dict = sp.read_input_file(input_file)
times, obs_specs = sp.get_obs_spec_and_times(io_dict)

# Creates the dictionary for each combination of parameters as defined in the input file
run_dictionaries = sp.create_runs_and_ids(fit_param_values)
# Grab the dictionary that corresponds to the model we want to look at
run_dictionary = run_dictionaries[int(model_number.split('_')[-1])]

Now that we've done this, we have access to everything that we've specified in the input file and the exact parameters that were used to calculate our model.  For example we can print out the line list or any of the stellar parameters that we want

In [ ]:
print(line_list)
print(run_dictionary)

The actual synthetic spectra that we've calculated live in a folder heirarchy that is structured as follows:

Output_folder_ID  (this is the output_directory specified in the input file)<br>
----> Model_XXXX    (there is one of these folders for each of the parameter combinations that you've calculated)<br>
--------> HeYY_CNOZZ  (there is one folder per abundance combination.  in this case we only have one)<br>
------------>hjdXXXXXXXXXXX_line  (there will be one of each line for each of the times specified in the input file)

Lets start by grabbing the HeI4471 line from our first model and plotting it.

In [ ]:
# define the path to the line
path_to_heI_line = output_folder + model_number + '/He%s_CNO%s/hjd0.00000000000_HEI4471.txt'%(He_abundance, CNO_abundance)
print(path_to_heI_line)

# plot the line
wavelength, flux = np.loadtxt(path_to_heI_line).T
fig, ax = plt.subplots()
ax.plot(wavelength, flux)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
plt.show()

Individual line profiles can be very useful in some circumstances, but often, plotting the whole spectrum at the same time is much more convenient.  Lets make a function that combines all of the line profiles from a single time step of a model into a spectrum

In [ ]:
def SPAMMS_lines_stitch(w1, f1, w2, f2):
    # To be used with SPAMMS spectral lines.  This function stitches together spectral lines by multiplying
    # normalized fluxes.  If there is no overlap region then the lines are just appended
    final_wave = []
    final_flux = []
    
    if w1[-1] < w2[0]:
        #if no overlap then
        final_wave.extend(w1)
        final_wave.extend(w2)
        final_flux.extend(f1)
        final_flux.extend(f2)
    else:
        # Define a wavelength array that covers both lines and interpolate both to the same wavelength array
        # Then multiply the fluxes together and return final wavelength and flux arrays
        
        final_wave = np.arange(w1[0], w2[-1]+0.001, 0.01)
        
        f1_interp = np.interp(final_wave, w1, f1, left=1.0, right=1.0)
        f2_interp = np.interp(final_wave, w2, f2, left=1.0, right=1.0)
        
        final_flux = f1_interp * f2_interp
        
        
    return final_wave, final_flux


def compile_SPAMMS_spectra(output_folder, model_number, He_abundance, CNO_abundance):
    # Load in the relevant information
    input_file = output_folder + 'input.txt'

    fit_param_values, abund_param_values, line_list, io_dict = sp.read_input_file(input_file)
    times, obs_specs = sp.get_obs_spec_and_times(io_dict)

    run_dictionaries = sp.create_runs_and_ids(fit_param_values)
    run_dictionary = run_dictionaries[int(model_number.split('_')[-1])]
    
    # define the path to the individual lines
    path_to_lines = output_folder + model_number + '/He%s_CNO%s/'%(He_abundance, CNO_abundance)
    
    # define our wavelength and flux lists where we will append the spectrum from each timestep
    wavelengths = []
    fluxes = []
    
    #loop through each time step
    for i in range(len(times)):

        line_wavelengths = []
        line_fluxes = []

        # loop through and grab each line
        for line in line_list:
            w,f = np.loadtxt(path_to_lines + 'hjd' + str(round(times[i], 13)).ljust(13, '0') + '_' + line + '.txt').T
            line_wavelengths.append(w)
            line_fluxes.append(f)
            
        # order the lines by wavelength to make stitching them together easier
        inds = np.array(np.argsort([i[0] for i in line_wavelengths]))

        # lets combine the lines to make a spectrum
        wavelength = line_wavelengths[inds[0]]
        flux = line_fluxes[inds[0]]
        for j in range(1, len(inds)):
            wavelength, flux = SPAMMS_lines_stitch(wavelength, flux, line_wavelengths[inds[j]], line_fluxes[inds[j]])

        wavelengths.append(wavelength)
        fluxes.append(flux)
    
    return wavelengths, fluxes


Lets pass all of the relevant arguments and plot the spectrum for this model.

In [ ]:
wavelengths, fluxes = compile_SPAMMS_spectra(output_folder, model_number, He_abundance, CNO_abundance)

# plot the spectrum
fig, ax = plt.subplots()
ax.plot(wavelengths[0], fluxes[0])
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
plt.show()

In [ ]:
# lets plot all of the rotation rates that we've calculated:

fig, ax = plt.subplots()
cmap = cm.plasma_r

for i in range(len(run_dictionaries)):
    model = 'Model_' + str(run_dictionaries[i]['run_id']).zfill(4)
    wavelengths, fluxes = compile_SPAMMS_spectra(output_folder, model, He_abundance, CNO_abundance)
    ax.plot(wavelengths[0], fluxes[0], c = cmap((i + 1) / (len(run_dictionaries)+1)), label = run_dictionaries[i]['v_crit_frac'])

ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
plt.legend(loc='best')
plt.show()

## Comparison between rotational distortion and spherical assumptions

Lets repeat the same SPAMMS run as above, but instead this time lets set the distortion method to spherical.  This will impose a spherical geometry for the system, which will mean that there are no surface variations due to the 3D geometry.  In the input file, lets change distortion to 'sphere' and change the name of the output folder to 'Demo_rapid_rotator_spherical'.  Then lets run SPAMMS and plot the output of the spherical models at each rotation rate

In [ ]:
output_folder = 'Outputs/Demo_rapid_rotator_spherical/'
input_file = output_folder + 'input.txt'

fit_param_values, abund_param_values, line_list, io_dict = sp.read_input_file(input_file)
times, obs_specs = sp.get_obs_spec_and_times(io_dict)

run_dictionaries = sp.create_runs_and_ids(fit_param_values)


# lets plot all of the rotation rates that we've calculated:

fig, ax = plt.subplots()
cmap = cm.plasma_r

for i in range(len(run_dictionaries)):
    model = 'Model_' + str(run_dictionaries[i]['run_id']).zfill(4)
    wavelengths, fluxes = compile_SPAMMS_spectra(output_folder, model, He_abundance, CNO_abundance)
    ax.plot(wavelengths[0], fluxes[0], c = cmap((i + 1) / (len(run_dictionaries)+1)), label = run_dictionaries[i]['v_crit_frac'])

ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
plt.legend(loc='best')
plt.show()

Lets compare an individual model from each.  These have all of the same stellar parameters, the only difference is the treatment of the geometry of the system.

In [ ]:
# initialize the plot
fig, ax = plt.subplots()

# rotational distortion 
wavelengths, fluxes = compile_SPAMMS_spectra('Outputs/Demo_rapid_rotator/', 'Model_0000', He_abundance, CNO_abundance)
ax.plot(wavelengths[0], fluxes[0], c = 'red', label = 'Roche')

# spherical
wavelengths, fluxes = compile_SPAMMS_spectra('Outputs/Demo_rapid_rotator_spherical/', 'Model_0000', He_abundance, CNO_abundance)
ax.plot(wavelengths[0], fluxes[0], c = 'black', label = 'Spherical')

ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
plt.legend(loc='best')
plt.show()